In [1]:
import os
import sys
from pandarallel import pandarallel
project_path = os.path.abspath(os.path.join(os.getcwd(), '..'))  # Go up one level
sys.path.append(project_path)
pandarallel.initialize(progress_bar=True, nb_workers=16)
import pandas as pd
from numpy.random import random
from utils.chatbot import QWENChatBot
from utils.chatbot_app import QWENapp
import time
import random
from concurrent.futures import ThreadPoolExecutor

INFO: Pandarallel will run on 16 workers.
INFO: Pandarallel will use standard multiprocessing data transfer (pipe) to transfer data between the main process and workers.


In [2]:
data = pd.read_excel('../processed_data/00_raw_data_filtered.xlsx')
data.head()

,id,数据类型,分析对象ID,分析对象,站点id,站点名称,标题,内容,视频封面内容识别,音频内容识别,...,视频时长,图片上的标签列表,笔记里的标签列表,笔记中艾特的用户列表,合作品牌,用户地址,认证简介,MCN机构名称,用户所获点赞和收藏数,classification
0,679260ba000000001800b050,电商笔记,58153241,分析对象1,146510,小红书,🇬🇧Recently plog🍛🥞💆🏻‍,1️⃣📍Drury 188-189 在holborn附近的网红brunch，点的开心果pan...,②小红书 HEYTEA|③POPHUART 延迟过万圣 POPMART Halloeen...,NaN,...,NaN,NaN,留学英国|发点生活边角料|清清日常库存|留学plog|伦敦brunch|labubu,NaN,NaN,广东广州,NaN,NaN,21000.0,0
1,6791f0c50000000029029680,电商笔记,58153241,分析对象1,146510,小红书,阿米的新年六位数购物开箱来了🎁,拖延症患者终于来开箱了，攒了一个多月的购物&护肤品分享，还有收到的礼物开箱，不算很齐全塞不下...,①UITTON LcUSvUmo 小红书 UITTON |②DUISVUITTON LO...,NaN,...,NaN,NaN,购物开箱分享|LV|lv村上隆|miumiu|chanel|新年礼物|乘光lumihood|...,NaN,NaN,中国,NaN,NaN,58000.0,0
2,6779f53f000000000b017934,电商笔记,58153241,分析对象1,146510,小红书,lv 村上隆限时店～还有2天！,一共三个店，黑彩，白彩，咖啡店～ 说实话三个店都非常可爱！但人真的好多！！（凑热闹本人+1）...,① LOUISVUITTON MURAKAMI|② ...,NaN,...,NaN,NaN,时尚潮人打卡地|随处走走就是潮流|潮流新地标,NaN,NaN,海外美国,NaN,NaN,71045.0,0
3,6784e9b10000000019034092,电商笔记,58153241,分析对象1,146510,小红书,终于到手了❗黑三彩carryall黑武士不要太好,开箱めLv村上隆包包■黑三彩Carryall 联名款黑武士羊皮carryall开箱分享 真的...,① 小红书 LOUISVUITON PARS,NaN,...,54.0,NaN,lv村上隆|LV黑三彩|lvcarryall|carryal新款|carryall|lv新款...,NaN,NaN,NaN,NaN,NaN,NaN,0
4,677953cd0000000013018d00,电商笔记,58153241,分析对象1,146510,小红书,Lv村上隆限定✖️白三彩卡包🆚黑三彩钱包,好可爱呀！兼并了美貌和实用#卡包推荐[话题]##钱包推荐[话题]# #lv[话题]# #l...,NaN,NaN,...,16.0,NaN,卡包推荐|钱包推荐|lv|lv村上隆联名|村上隆|lv白三彩|lv黑三彩,NaN,NaN,日本 东京,NaN,NaN,600.0,0


In [ ]:
def qwen_tagging(comment):
    chatbot = QWENChatBot('你是一个专业的数据标注助手，专注于从社交媒体用户生成内容（UGC）中提取和标注特定的信息。你需要针对小红书上关于LVMH和村上隆合作新系列的反馈分析项目提取结构化信息')
    prompt = """

任务：
请根据以下用户生成内容（UGC）评论，按照本项目预定义的标签标准，提取与LVMH × 村上隆合作新品系列内容关联的结构化信息。所有输出必须符合 JSON 格式规范，确保输出结构完全一致.

1.确保输出内容清晰明确，不能出现模糊或不确定的情况。
2.JSON 结构中的每个字段都必须严格按照给定选项填写。如果某个选项没有涉及到，请使用空字符串 "" 或空列表 [] 表示。
3.对于有预定义分类的字段，请根据以下规则填写：
年龄范围（Age_Range）: 只能填写 "18岁以下"、"18-24岁"、"25-34岁"、"35-44岁"、"45岁以上" 或 ""（当无法判断时）。
性别（Gender）: 只能填写 "男性"、"女性" 或 ""（当无法判断时）。
城市等级（City_Tier）: 只能填写 "一线"、"二线"、"三线及以下" 或 ""（当无法判断时）。
4.对于 购买需求（Purchase_Purpose） 和 购买情境（Purchase_Situation）：
购买需求（Purchase_Purpose）:根据帖子内容，对用户为什么要购买/购买LV和村上隆合作新系列的动机进行分类，或 ""（当无法判断或未购买LV产品时）。
购买情景（Purchase_Situation）：根据帖子内容，对购买的场景进行分类，或 ""（当无法判断或未购买LV产品时）。
若评论未体现相关信息，则填写 ""。
5.对于提到的产品（Product_Mention）:
提取用户提到的LV和村上隆合作新系列的产品名称，或 []（当无法判断时）。
6.对于提到的产品（Competitor）:
提取用户提到的LV以外的竞争品牌名称，或 []（当无法判断时）。
7.对于用户的情感（Emotion）：
根据帖子内容和预期判断用户对lv联名系列的情感，或""当无法判断时）。
8.对于用户的语气（Tone）：
根据帖子的内容，总结说话语气，如成熟稳重，活泼俏皮等，或""（当无法判断时）。
9.关键词提取（Keywords）: 对评论内容中提到的 设计元素（Design_Elements）、设计风格（Design_Style） 和 功能性（Functionality） 关键词分别分类。
设计元素（Design_Elements）： 包括颜色（如"红色"）、尺码（如"大号"）、以及新系列的饰品细节（如"樱花吊坠"）等。
设计风格（Design_Style）： 包括了总体风格词描述，比如"复古"、"青春"、"可爱"等。
功能性（Functionality）： 聚焦用户对产品功能性的具体评价以及期待，例如"有内袋"、"自重轻"、"百搭"等。
其他关键词（Other）： 包括无法归类至其他子类别但与新品系列相关的重要关键词列表。

输出 JSON 结构：
{
    "Age_Range": str, 
    "Gender": str,  
    "City_Tier": str,  
    "Purchase_Purpose": str,  
    "Purchase_Situation": str,  
    "Product_Mention":List[str],
    "Competitor":List[str],
    "Emotion":str,
    "Tone": str,
    "Keywords": {
        "Design_Elements": List[str],  
        "Design_Style": List[str],  
        "Functionality": List[str], 
        "Other": List[str]  
    }
}


示例输入：
用户生成内容评论：

“超级喜欢这次村上隆联名系列的樱花吊坠设计，真是绝美！特别是粉色的颜色超少女心，大小也很适合日常出街。像我这种时尚控完全爱了！而且还可以斜挎，直接入手了限量款！”

示例输出：

{
    "Age_Range": "25-34岁",
    "Gender": "女性",
    "City_Tier": "一线",
    "Purchase_Purpose": "追求独特设计",
    "Purchase_Situation": "日常出街",
    "Product_Mention": ["樱花吊坠", "限量款"],
    "Competitor": [],
    "Emotion": "喜欢",
    "Tone": "活泼俏皮",
    "Keywords": {
        "Design_Elements": ["樱花吊坠", "粉色", "大小适合"],
        "Design_Style": ["少女心", "时尚"],
        "Functionality": ["斜挎"],
        "Other": ["村上隆", "联名系列"]
    }
}

你只能按照格式返回且只返回 JSON，如果你无法提取信息，按照规则返回所要求格式的 JSON即可,输出中不能出现多余的解释性文字、描述性文本或格式修饰符，例如：“根据提供的评论内容：”或“```json”。

请按照以下评论内容生成 JSON 标注：

评论内容：""" + f"{comment}"
    try:
        # Call the chatbot API and add random delay for throttling
        response = chatbot.ask(prompt)
        time.sleep(random.uniform(0.2, 0.8))  # Avoid rate limits
        return response
    except Exception as e:
        print(f"Error processing comment: {comment}\nException: {e}")
        return "{}"  # Return empty JSON on failure
chatbot = QWENChatBot(system_message=system_message)


# 使用线程池处理 DataFrame 数据
def classify_post(content: str) -> str:
    """
    A process-safe function to classify a single post using the chatbot.
    """
    try:
        if pd.isnull(content):  # Handle NaN (null) input
            return "未明确提及"
        return chatbot.ask(content)  # Ensure this returns only the classification
    except Exception as e:
        return f"Error: {e}"  # Return error string
data_tagged['Purpose_Category'] = data_tagged['Purchase_Purpose'].parallel_apply(lambda x:classify_post(x))

data['raw_tagging'] = data['内容'].parallel_apply(lambda x:qwen_tagging(x))

'''def apply_parallel(column_data, max_workers=5):
    """
    Parallelizes processing over a single column using ThreadPoolExecutor.
    """
    with ThreadPoolExecutor(max_workers=max_workers) as executor:
        print("Starting parallel processing...")
        # Apply the function to each comment in parallel
        results = list(executor.map(qwen_tagging, column_data))
    return results
if 'raw_tagging' not in data.columns:
    data['raw_tagging'] = '0'
data["raw_tagging"] = apply_parallel(data["内容"].tolist(), max_workers=16)'''
data[['内容','raw_tagging']]


In [3]:
system_message = """
#角色：
你是一个专业的数据标注助手，专注于从社交媒体用户生成内容（UGC）中提取和标注特定的信息。lv和村上隆在2003年第一次合作联名，时隔20多年在2025年再次合作，你需要针对小红书上关于LVMH和村上隆合作新系列的反馈分析项目提取结构化信息。

任务：
请根据以下用户生成内容（UGC）评论，按照本项目预定义的标签标准提取信息，输出必须符合 JSON 格式规范，确保输出结构完全一致.

1.确保输出内容清晰明确，不能出现模糊或不确定的情况。
2.JSON 结构中的每个字段都必须严格按照给定选项填写。如果某个选项没有涉及到，请使用空字符串 "" 或空列表 [] 表示。
3.对于有预定义分类的字段，请根据以下规则填写：
年龄范围（Age_Range）: 根据lv和村上隆两次合作时间，以及贴文内容提取或推断用户年龄（例如提到自身相关信息/家人等），只能填写 "18岁以下"、"18-24岁"、"25-34岁"、"35-44岁"、"45岁以上" 或 ""（当无法判断时）。
性别（Gender）: 只能填写 "男性"、"女性" 或 ""（当无法判断时）。
城市等级（City_Tier）: 只能填写 "一线"、"二线"、"三线及以下" 或 ""（当无法判断时）。
4.对于 购买需求（Purchase_Purpose） 和 购买情境（Purchase_Situation）：
购买需求（Purchase_Purpose）:根据帖子内容，对用户为什么要购买/购买LV和村上隆合作新系列的动机进行分类，或 ""（当无法判断或未购买LV产品时）。
购买情景（Purchase_Situation）：根据帖子内容，对购买的场景进行分类，或 ""（当无法判断或未购买LV产品时）。
若评论未体现相关信息，则填写 ""。
5.对于提到的产品（Product_Mention）:
提取用户提到的LV和村上隆合作新系列的产品名称，或 []（当无法判断时）。
6.对于提到的产品（Competitor）:
提取用户提到的LV以外的竞争品牌名称，或 []（当无法判断时）。
7.对于用户的情感（Emotion）：
根据帖子内容和预期判断用户对lv联名系列的情感，或""当无法判断时）。
8.对于用户的语气（Tone）：
根据帖子的内容，总结说话语气，如成熟稳重，活泼俏皮等，或""（当无法判断时）。
9.对于历史购买（Past_Purchase)：
跟据帖子内容判断用户此前是否购买过lv和村上隆联名系列，如过有相关购买返回"是"，没有历史购买或为提及返回"否"
10.关键词提取（Keywords）: 对评论内容中提到的 设计元素（Design_Elements）、设计风格（Design_Style） 和 功能性（Functionality） 关键词分别分类。
设计元素（Design_Elements）： 包括颜色（如"红色"）、尺码（如"大号"）、以及新系列的饰品细节（如"樱花吊坠"）等。
设计风格（Design_Style）： 包括了总体风格词描述，比如"复古"、"青春"、"可爱"等。
功能性（Functionality）： 聚焦用户对产品功能性的具体评价以及期待，例如"有内袋"、"自重轻"、"百搭"等。
其他关键词（Other）： 包括无法归类至其他子类别但与新品系列相关的重要关键词列表。

输出 JSON 结构：
{
    "Age_Range": str, 
    "Gender": str,  
    "City_Tier": str,  
    "Purchase_Purpose": str,  
    "Purchase_Situation": str,  
    "Product_Mention":List[str],
    "Competitor":List[str],
    "Emotion":str,
    "Tone": str,
    "Past_Purchase": str,
    "Keywords": {
        "Design_Elements": List[str],  
        "Design_Style": List[str],  
        "Functionality": List[str], 
        "Other": List[str]  
    }
}


示例输入：
用户生成内容评论：

“超级喜欢这次村上隆联名系列的樱花吊坠设计，真是绝美！特别是粉色的颜色超少女心，大小也很适合日常出街。像我这种时尚控完全爱了！而且还可以斜挎，直接入手了限量款！”

示例输出：

{
    "Age_Range": "25-34岁",
    "Gender": "女性",
    "City_Tier": "一线",
    "Purchase_Purpose": "追求独特设计",
    "Purchase_Situation": "日常出街",
    "Product_Mention": ["樱花吊坠", "限量款"],
    "Competitor": [],
    "Emotion": "喜欢",
    "Tone": "活泼俏皮",
    "Past_Purchase": "否",
    "Keywords": {
        "Design_Elements": ["樱花吊坠", "粉色", "大小适合"],
        "Design_Style": ["少女心", "时尚"],
        "Functionality": ["斜挎"],
        "Other": ["村上隆", "联名系列"]
    }
}

你只能按照格式返回且只返回 JSON，如果你无法提取信息，按照规则返回所要求格式的 JSON即可,输出中不能出现多余的解释性文字、描述性文本或格式修饰符，例如：“根据提供的评论内容：”或“```json”。
"""
chatbot = QWENChatBot(system_message = system_message)


# 使用线程池处理 DataFrame 数据
def classify_post(content: str) -> str:
    """
    A process-safe function to classify a single post using the chatbot.
    """
    try:
        response = chatbot.ask(content) 
        return response  # Ensure this returns only the classification
    except Exception as e:
        return f"Error: {e}"  # Return error string
data['raw_tagging'] = data['内容'].parallel_apply(lambda x:classify_post(x))



'''def apply_parallel(column_data, max_workers=5):
    """
    Parallelizes processing over a single column using ThreadPoolExecutor.
    """
    with ThreadPoolExecutor(max_workers=max_workers) as executor:
        print("Starting parallel processing...")
        # Apply the function to each comment in parallel
        results = list(executor.map(qwen_tagging, column_data))
    return results
if 'raw_tagging' not in data.columns:
    data['raw_tagging'] = '0'
data["raw_tagging"] = apply_parallel(data["内容"].tolist(), max_workers=16)'''
data[['内容','raw_tagging']]


,内容,raw_tagging
0,1️⃣📍Drury 188-189 在holborn附近的网红brunch，点的开心果pan...,"{\n ""Age_Range"": ""18-24岁"",\n ""Gender"": ""..."
1,拖延症患者终于来开箱了，攒了一个多月的购物&护肤品分享，还有收到的礼物开箱，不算很齐全塞不下...,"{\n ""Age_Range"": ""25-34岁"",\n ""Gender"": ""..."
2,一共三个店，黑彩，白彩，咖啡店～ 说实话三个店都非常可爱！但人真的好多！！（凑热闹本人+1）...,"{\n ""Age_Range"": ""18-24岁"",\n ""Gender"": ""..."
3,开箱めLv村上隆包包■黑三彩Carryall 联名款黑武士羊皮carryall开箱分享 真的...,"{\n ""Age_Range"": """",\n ""Gender"": """",\n ..."
4,好可爱呀！兼并了美貌和实用#卡包推荐[话题]##钱包推荐[话题]# #lv[话题]# #l...,"{\n ""Age_Range"": """",\n ""Gender"": ""女性"",\n..."
...,...,...
2622,800年没买trainer的我陪母上大人逛街顺利蹭到一双[派对R][吐舌头H][耶R] #鞋...,"{\n ""Age_Range"": """",\n ""Gender"": """",\n ..."
2623,村上隆太阳花熊猫。还有比这个组合更合适的包给我们花花吗[派对R][派对R][派对R]#LV ...,"{\n ""Age_Range"": """",\n ""Gender"": """",\n ..."
2624,#lv村上隆快闪店 太出片啦！！！今天下午去的，除了咖啡店拍的时间久点外，其他都很快，地方也...,"{\n ""Age_Range"": """",\n ""Gender"": """",\n ..."
2625,lv村上隆联名这次出的speedy系列，比speedy30小的只有speedynano，有看...,"{\n ""Age_Range"": """",\n ""Gender"": """",\n ..."


In [10]:
data[['内容','raw_tagging']].to_excel('../processed_data/01_tagged_raw.xlsx', index=False)